# ***Axiom AI Demo***

In [ ]:
#@title <font class="markdown-google-sans">Install</font>
import os
os.chdir("/content/")
!git clone https://github.com/kubinka0505/axiom
os.chdir("axiom/Files")
!python setup.py install

In [ ]:
#@title <font class="markdown-google-sans">Mount drive</font>
from google.colab import drive
drive.mount("/content/drive")

# <font class="markdown-google-sans">Training</font>

In [ ]:
#@title ## <font class="markdown-google-sans">Preprocess dataset</font>
#@markdown > Optional.
#@markdown After preprocessing is finished, path for train/infer is adjusted accordingly.

frequency_step = 100 #@param {type: "number"}
dataset_directory = "/content/drive/MyDrive/axiom/dataset/1" #@param {type: "string"}
output_directory = "/content/axiom_datasets/1_outs" #@param {type: "string"}
extension = "WAV" #@param ["WAV", "FLAC"]
overwrite = False #@param {type: "boolean"}

#@markdown &nbsp;

show_help = False #@param {type: "boolean"}

help_str = "-h" if show_help else ""
overwrite_str = "-f" if overwrite else ""

os.chdir("/content/axiom/Files/src/ai")
!python prep.py \
	-i {dataset_directory} \
	-o {output_directory} \
	{overwrite_str} \
	-fmt {extension} \
	-k {frequency_step} \
	{help_str}

dataset_directory = output_directory

In [ ]:
#@title ## <font class="markdown-google-sans">Begin training</font>
%load_ext tensorboard
%tensorboard --logdir /content/logs

epochs = 100 #@param {type: "number"}

#@markdown &nbsp;

#@markdown #### <font class="markdown-google-sans">Dataset directory</font>
dataset_directory_from_preprocess = True #@param {type: "boolean"}
dataset_directory = "/content/drive/MyDrive/axiom/dataset/1" #@param {type: "string"}
model_destination = "/content/drive/MyDrive/axiom/models" #@param {type: "string"}

#@markdown &nbsp;

#@markdown #### <font class="markdown-google-sans">Training settings</font>
save_each_epoch = 1 #@param {type: "slider", min: 1, max: 25, step: 1}
eval_each_epoch = 1 #@param {type: "slider", min: 1, max: 10, step: 1}
model_architecture = "v2" #@param ["v1", "v2"]

#-=-=-=-#

if dataset_directory_from_preprocess:
	dataset_directory = output_directory

model_architecture = "".join(x for x in model_architecture if x.isdigit()

!python model.py train \
	-i {dataset_directory} \
	-e {epochs} \
	-see {save_each_epoch} \n	-eee {eval_each_epoch} \n	-a {model_architecture} \
	-o /content/ \
	-logs /content/logs

# <font class="markdown-google-sans">Inference</font>

In [ ]:
#@title <font class="markdown-google-sans">Begin inference</font>
#@markdown If any of parameters below are directories, fetches latest applicable files from them.

#@markdown If any of parameters below are empty, raises `files.upload()`
from google.colab import files

input_file = "" #@param {type: "string"}
model_path = "/content/drive/MyDrive/axiom/models" #@param {type: "string"}

os.chdir("/content/")
if not input_file:
	input_file = files.upload()
	input_file = list(input_file.keys())[0]

if not model_path:
	model_path = files.upload()
	model_path = list(model_path.keys())[0]

os.chdir("/content/axiom/src")
!python model.py infer \
	-i {input_file}
	-m {model_file}